In [77]:
import pymatgen as pmg
from pymatgen.io.vasp.outputs import Outcar
from pymatgen.electronic_structure.core import Spin
import numpy as np

from os.path import expanduser
home = expanduser('~')

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


def format_ladder(orbitals, occu_matrix, spin=None):
    ladder = np.array(sorted(zip(np.diag(occu_matrix), orbitals), reverse=True))
    occu_ladder = ladder[:,0]
    orb_ladder = ladder[:,1]
    if spin == 1:
        row1 = '/\ Orbital:   '
        row2 = '/\ Occupation:'
    elif spin == -1:
        row1 = '\/ Orbital:   '
        row2 = '\/ Occupation:'
    for i in range(len(occu_ladder)):
        row1 = row1 + f'\t{orb_ladder[i]}'
        row2 = row2 + f'\t{occu_ladder[i]}'
    return row1 + '\n' + row2


def display_occu(odm, orbitals, order=None, print_ladder=True):
    up = odm[Spin.up]
    down = odm[Spin.down]
    
    ### Write header
    header = '\t'
    if order == None: order = range(5)
    for i in order:
        header = f'{header}{orbitals[i]}\t\t'
    print(header, end='')
    
    ### Write each row
    for i in order:
        print(f'\n{orbitals[i]}', end='\t')
        for j in order:
            entries = [np.round(up[i][j], 2), np.round(down[i][j], 2)]
            for k in range(2):
                if entries[k] == 0:
                    entries[k] = '----'
            print(f'{entries[0]} / {entries[1]}', end='\t')
    print('\n')
    if print_ladder:
        print(format_ladder(orbitals, up, spin=1))
        print()
        print(format_ladder(orbitals, down, spin=-1))
#         up_ladder = zip(np.diag(up), orbitals)
#         up_ladder =  sorted(up_ladder, reverse=True)
#         print(np.array(up_ladder).T)
#         print('\n')
        
#         dn_ladder = zip(np.diag(down), orbitals)
#         dn_ladder =  sorted(dn_ladder, reverse=True)
#         print(np.array(dn_ladder).T)
#         print('\n')
    

def print_occupations(iters, orbitals, order=None, atom_num=0):
    odms = []
    for i, iter_1 in enumerate(iters):
            label = f'{iter_1}'
            outcar = Outcar(f'{PREFIX}-{label}{SUFFIX}')
            outcar.read_onsite_density_matrices()
            odm = outcar.data['onsite_density_matrices'][-4:]
            odms.append(odm)
            print(f'##### {iter_1} #####')
            display_occu(odm[atom_num], orbitals, order=order)
            print()
            
            
PREFIX = home + '/Projects/BaCoS2/correct_mag/estruc/OUTCAR'
SUFFIX = ""

In [70]:
### Unrotated cells (45deg rotation along z axis from alignment)
################################################################

# ITERS_1 = ['U0', 'exp-U0', 'U2', 'exp-U2',  'U2.2', 'U4', 'exp-U4']
ITERS_1 = ['U0', 'U2', 'U2.5', 'U3', 'U4']

LABELS_1 = ITERS_1

orbitals_fancy = ['$d_{x^2 - y^2}$', '$d_{yz}$', '$d_{z^2 - r^2}$', '$d_{xz}$', '$d_{xy}$']
orbitals = ['d_x2y2', 'd_yz', 'd_z2-r2', 'd_xz', 'd_xy']
order = [4, 1, 3, 2, 0]

print_occupations(ITERS_1, orbitals, order)

##### U0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2y2		
d_xy	0.9 / 0.86	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.87 / 0.68	---- / ----	---- / ----	---- / ----	
d_xz	---- / ----	---- / ----	0.86 / 0.69	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / ----	---- / ----	0.8 / 0.59	---- / ----	
d_x2y2	---- / ----	---- / ----	---- / ----	---- / ----	0.72 / 0.47	

/\ Orbital:   	d_xy	d_yz	d_xz	d_z2-r2	d_x2y2
/\ Occupation:	0.8992	0.8671	0.8583	0.7993	0.7224

\/ Orbital:   	d_xy	d_xz	d_yz	d_z2-r2	d_x2y2
\/ Occupation:	0.8568	0.6941	0.6775	0.5937	0.4681

##### U2 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2y2		
d_xy	0.93 / 0.85	---- / ----	---- / ----	---- / -0.17	---- / ----	
d_yz	---- / ----	0.93 / 0.88	---- / -0.01	---- / ----	---- / ----	
d_xz	---- / ----	---- / -0.01	0.94 / 0.22	---- / ----	---- / ----	
d_z2-r2	---- / -0.17	---- / ----	---- / ----	0.92 / 0.42	---- / ----	
d_x2y2	---- / ----	---- / ----	---- / ----	---- / ----	0.92 / 0.31	

/\ Orbital:   	d_xz	d_xy	d_yz	

In [78]:
### Rotated cells -- properly aligned
#####################################

ITERS_rot = ['rot-U0', 'rot-U3', 'rot-U4']

orbitals_fancy = ['$d_{xy}$', '$d_{yz}$', '$d_{z^2 - r^2}$', '$d_{xz}$', '$d_{x^2 - y^2}$']
orbitals = ['d_xy', 'd_yz', 'd_z2-r2', 'd_xz', 'd_x2y2']
order = [0, 1, 3, 2, 4]

print_occupations(ITERS_rot, orbitals, order, atom_num=3)

##### rot-U0 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2y2		
d_xy	0.89 / 0.85	---- / ----	---- / ----	---- / ----	---- / ----	
d_yz	---- / ----	0.86 / 0.7	---- / 0.01	---- / ----	---- / ----	
d_xz	---- / ----	---- / 0.01	0.86 / 0.7	---- / ----	---- / ----	
d_z2-r2	---- / ----	---- / ----	---- / ----	0.79 / 0.6	---- / ----	
d_x2y2	---- / ----	---- / ----	---- / ----	---- / ----	0.71 / 0.48	

/\ Orbital:   	d_xy	d_yz	d_xz	d_z2-r2	d_x2y2
/\ Occupation:	0.8937	0.8561	0.8561	0.7894	0.7139

\/ Orbital:   	d_xy	d_yz	d_xz	d_z2-r2	d_x2y2
\/ Occupation:	0.8542	0.6959	0.6959	0.6038	0.4804

##### rot-U3 #####
	d_xy		d_yz		d_xz		d_z2-r2		d_x2y2		
d_xy	0.94 / 0.82	---- / ----	---- / ----	---- / 0.27	---- / ----	
d_yz	---- / ----	0.95 / 0.55	---- / 0.38	---- / ----	---- / ----	
d_xz	---- / ----	---- / 0.38	0.95 / 0.55	---- / ----	---- / ----	
d_z2-r2	---- / 0.27	---- / ----	---- / ----	0.95 / 0.28	---- / ----	
d_x2y2	---- / ----	---- / ----	---- / ----	---- / ----	0.97 / 0.29	

/\ Orbital:   	d_x2y2	d_z2-

In [8]:
### Checking diagonalization -- rotated cell shows that the orbital ordering above is correct
### the yz and xz are indeed degenerate 
### and the t2g are in fact inverted

m = odms[5][0][Spin.down]
print(np.array(m))
eig = np.linalg.eig(m)
print()
print(np.round(eig[0],3))
print(np.round(eig[1],3))

[[ 0.2601 -0.      0.      0.     -0.    ]
 [-0.      0.1253 -0.     -0.      0.    ]
 [ 0.     -0.      0.2807 -0.      0.3009]
 [ 0.     -0.     -0.      0.9392  0.    ]
 [-0.      0.      0.3009  0.      0.7998]]

[0.143 0.938 0.26  0.125 0.939]
[[ 0.     0.     1.     0.     0.   ]
 [ 0.     0.     0.     1.     0.   ]
 [-0.909 -0.416  0.     0.     0.   ]
 [ 0.     0.     0.     0.     1.   ]
 [ 0.416 -0.909  0.     0.     0.   ]]


In [ ]:
m = odms[1][0][Spin.down]
print(m)
eig = np.linalg.eig(m)
print()
print(np.round(eig[0],2))
print(np.round(eig[1],2))